In [204]:
from pulp import *
import pulp
import pandas as pd
import numpy as np
import random

In [205]:
# Read the Excel file into a DataFrame
df = pd.read_excel('file.xlsx')

# Convert the DataFrame to a NumPy array
data = df.values
products = []

for row in data:
    predictedProfits = row[0]
    predictedProductQuantities = row[1]
    dimension = row[2]
    productName = row[3]

    product = {'predictedProfits': predictedProfits, 'predictedProductQuantities': predictedProductQuantities, 'dimension': dimension, 'productName': productName}
    products.append(product)

print(products)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1'}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2'}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3'}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4'}]


In [206]:
def calculate_profit(products):
    for product in products:
        profit = product['predictedProfits'] / product['predictedProductQuantities']
        product['productProfit'] = profit
    return products

In [207]:
productTest = calculate_profit(products)
print(productTest)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]


In [208]:
# Creation of the ILP model 
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

In [209]:
# Define the decision variables
decisionVariables = []
def createDictionaryWithDecisionVariable():
    for product in products:
        decisionVariable = {product['productName']:  pulp.LpVariable(product['productName'], lowBound=0, cat='Integer')}
        decisionVariables.append(decisionVariable)
        
createDictionaryWithDecisionVariable()
print(decisionVariables)

[{'p1': p1}, {'p2': p2}, {'p3': p3}, {'p4': p4}]


In [210]:
# function to calculate objective function

def objectiveFunction(products):
    model = 0
    count = 0
    for product in products:
        model = model + (product['predictedProfits'] * decisionVariables[count][product['productName']])
        count = count + 1
    return model

#objective function
model += objectiveFunction(products)
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [211]:
#add user input for the multiple variables

minTotalAmount = int(input("Please insert minimum total amount of products: "))
maxTotalAmount = int(input("Please insert maximum  total amount of products: "))

#create two list with min and max amount per product by user insert

minAmountProduct = int(input("Please insert minimum  amount of products : "))
maxAmountProduct  = int(input("Please insert maximum  amount of products : "))

minProductVariety = int(input("Please insert minimum number of product variety: "))
maxProductVariety  = int(input("Please insert maximum  number of product variety: "))

storeDimension  = int(input("Please insert store dimension: "))

Please insert minimum total amount of products: 10
Please insert maximum  total amount of products: 1000
Please insert minimum  amount of products : 10
Please insert maximum  amount of products : 100
Please insert minimum number of product variety: 4
Please insert maximum  number of product variety: 10
Please insert store dimension: 10000000000000000


In [212]:
# Define the constraints

 # function to calculate variety of products
def get_variety(products):
    # Create an empty set to store the product names
    product_set = []
    
    # Iterate over the products
    for product in products:
        # Get the product name and add it to the set
        product_name = product['productName']
        product_set.append(product_name)
        
    # Return the length of the set
    return len(product_set)

print(get_variety(products))
variety=get_variety(products)

#see why it results in Infeasible
# model += variety <= maxProductVariety
# model += variety >= minProductVariety


#restriction to total amount of products

def maxMinTotalProductsRestriction(products):
    restrictFunction = 0
    count = 0
    for product in products:
        restrictFunction = restrictFunction + decisionVariables[count][product['productName']]
        count = count + 1
    return restrictFunction
    
model += maxMinTotalProductsRestriction(products) <= maxTotalAmount #restricao de quantidade maximo total de produtos
model += maxMinTotalProductsRestriction(products) >= minTotalAmount #restricao de valor minimo total de produtos

# variety of products

#restriction to maximum quantity of products
def maxMinQuantityProductsRestriction(products,model):
    count = 0
    for product in products:
        model += decisionVariables[count][product['productName']] <= maxAmountProduct
        model += decisionVariables[count][product['productName']] >= minAmountProduct
        count = count + 1
    
maxMinQuantityProductsRestriction(products, model)

print(maxAmountProduct)
print(minAmountProduct)


def maxDimensionProductsRestriction(products):
    count = 0
    restrictFunction = 0
    for product in products:                            
        restrictFunction = restrictFunction + (product['dimension'] * decisionVariables[count][product['productName']])
        count = count +1
    return restrictFunction

print(maxDimensionProductsRestriction(products))
#restricao de dimensões de produtos, menor que dimensao todal da loja
model += maxDimensionProductsRestriction(products) <= storeDimension 


4
100
10
5*p1 + 2*p2 + p3 + 20*p4


In [213]:
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
SUBJECT TO
_C1: p1 + p2 + p3 + p4 <= 1000

_C2: p1 + p2 + p3 + p4 >= 10

_C3: p1 <= 100

_C4: p1 >= 10

_C5: p2 <= 100

_C6: p2 >= 10

_C7: p3 <= 100

_C8: p3 >= 10

_C9: p4 <= 100

_C10: p4 >= 10

_C11: 5 p1 + 2 p2 + p3 + 20 p4 <= 1e+16

VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [214]:
# Solve the problem
status = model.solve()

In [215]:
print(pulp.LpStatus[status])

Optimal


In [216]:
#print(pulp.value(p1), pulp.value(model.objective))

In [217]:
# Print the solution
#print("Solution:")
#print("x =", x.value())
#print("y =", y.value())

In [218]:
def printValuesReturned():
    for v in model.variables():
        try:
            print(v.name,"=", v.value())
        except:
            print("error couldnt find value")
            
printValuesReturned()
print(len(products))
print(products)
print(products[0]['dimension'])

p1 = 100
p2 = 100
p3 = 100
p4 = 100
4
[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]
5


In [219]:
# Genetic Algorithm

productsList = []
print(productsList)
 
for product in products:
    productsList.append((product['productName'], product['predictedProductQuantities'], product['predictedProfits'], product['dimension']))
for p in productsList:
    print(p)

[]
('p1', 10, 100, 5)
('p2', 20, 200, 2)
('p3', 30, 300, 1)
('p4', 1, 10, 20)


In [220]:
# Functions to validate restrictions
# return 1 if restriction is repected else 0

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)
    if (somaTotalAmountIndividuo >= minTotalAmount) and (somaTotalAmountIndividuo <= maxTotalAmount):
        return 1
    else:
        return 0
    
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

#Dimension Restriction
def validateDimension(individuo, products):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimension):
        return 0
    else:
        return 1

In [221]:
# Create First Population

population = []

def firstpopulation(products):
    
    for i in range(100):
        
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        for j in range(len(products)):
            
            quantity = random.randint(0, maxAmountProduct)
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution, products)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            population.append(solution)
            
    return population

In [222]:
firstpopulation(products)

print(len(firstpopulation(products)))
print(population)

131
[[81, 27, 25, 71], [26, 43, 99, 72], [70, 96, 40, 96], [68, 82, 43, 47], [56, 59, 36, 47], [88, 45, 83, 69], [52, 77, 32, 55], [44, 43, 98, 61], [43, 55, 64, 36], [47, 32, 43, 95], [50, 58, 92, 75], [92, 67, 91, 41], [68, 75, 80, 82], [67, 29, 23, 72], [11, 57, 77, 68], [58, 62, 95, 94], [49, 92, 75, 40], [83, 13, 41, 26], [18, 99, 98, 68], [74, 92, 12, 66], [37, 23, 83, 71], [65, 45, 90, 40], [68, 99, 17, 85], [72, 33, 57, 56], [94, 47, 71, 56], [15, 18, 62, 12], [88, 67, 75, 61], [90, 97, 81, 53], [30, 67, 33, 52], [19, 31, 62, 94], [52, 87, 76, 19], [15, 40, 46, 62], [98, 62, 49, 48], [72, 88, 61, 75], [86, 67, 13, 44], [61, 87, 44, 25], [26, 39, 27, 99], [33, 88, 56, 75], [84, 77, 97, 56], [70, 79, 52, 29], [98, 16, 86, 21], [79, 87, 79, 76], [78, 77, 92, 71], [15, 71, 39, 29], [78, 84, 76, 16], [63, 47, 71, 73], [47, 44, 42, 10], [19, 98, 33, 99], [85, 72, 21, 99], [43, 31, 60, 90], [57, 64, 90, 15], [21, 14, 97, 74], [58, 69, 41, 10], [39, 87, 80, 93], [19, 83, 66, 46], [22, 

In [225]:
# Create Fitness function to give the best solution

arraySomaProfits = []
arrayFitProfitAndQuantity = []
finalPopulation = []

# if this element is 1 the first tax of best individuos is added to final population
elementToGetFirstBestIndividuos = 0

taxNoCrossover = 0.10

def createFitnessFunction(population, products):
    
    for person in population:
        
        soma = 0
        contador = 0
        
        for productQuantity in person:
            
            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1
            
        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.append(auxVar)
        arraySomaProfits.append(soma)
        
    array = np.array(arraySomaProfits)
    array[::-1].sort()
    
    # get out taxNoCrossover to final population
    
    numberElementsNoCrossover = len(population) * taxNoCrossover
    
    for i in range(round(numberElementsNoCrossover)):
        
        for element in arrayFitProfitAndQuantity:
            
            if element['FitProfit'] == array[i]:
                finalPopulation.append(element['FitQuantities'])
                population.remove(element['FitQuantities'])
            
    return array

fitSolutions = createFitnessFunction(population, products)
#maxFitSolution = np.max(createFitnessFunction(population, products))

print(fitSolutions)
print(maxFitSolution)
print(finalPopulation)
print(len(population) )

[57520 53460 53230 51680 51510 50310 49760 47640 47560 46680 46620 46230
 46200 46140 45940 45840 45650 45590 45310 44980 44950 43850 43510 43390
 43270 43010 42900 42610 42440 42430 42310 42280 41620 40750 40660 40650
 40540 40460 40260 40060 39820 39160 39010 38760 38690 38530 38450 38320
 38080 37990 37940 37730 37530 37530 37380 36950 36880 36830 36570 36400
 36310 36280 35530 35240 34860 34740 34680 34180 33910 33040 32920 32910
 32720 32550 32390 32370 32000 31520 31460 31110 30900 30750 30720 30530
 30220 30190 30060 29990 29440 29400 29250 29240 28790 28780 28670 28590
 27720 27690 27640 27550 27330 27070 26820 26340 26200 26170 25810 25720
 25180 24950 23920 23820 23460 23110 22640 21850 21710 21450 20770 20500
 20120 19490 19290 18690 17140 16620 16350 15720 13550 12550 11090]
27900
[[87, 99, 96, 22], [84, 77, 97, 56], [90, 97, 81, 53], [18, 99, 98, 68], [78, 77, 92, 71], [92, 67, 91, 41], [79, 87, 79, 76], [58, 62, 95, 94], [78, 84, 76, 16], [97, 38, 97, 28], [68, 75, 80, 82

In [228]:
print(population[1])
print(random.randint(0, len(population[0])))

[26, 43, 99, 72]
3


In [261]:
# Function to crossover

def crossover(population):
    
 #   while len(population) > 1:
        
        firstPai = population[0]
        secondPai = population[1]
        print(firstPai)
        print(secondPai)
        crossoverPoint = random.randint(0, len(population[0]))
        for i in range(crossoverPoint):
            aux = firstPai[i]
            print(aux)
            firstPai[i] = secondPai[i]
            print(firstPai[i])
            secondPai[i] = aux
            print(secondPai[i])
            
        print(firstPai)
        print(secondPai)
 
crossover(population)

[81, 43, 25, 71]
[26, 27, 99, 72]
81
26
81
43
27
43
25
99
25
71
72
71
[26, 27, 99, 72]
[81, 43, 25, 71]


In [222]:
import random


def optimize_product_quantities(productsList, maxTotalQuantity, maxProductQuantity, storeDimension):
    # products is a list of tuples, where each tuple contains the following information about a product:
    # (name, quantity, price, dimension)

    # Create a population of random solutions

    # population = []
    # for i in range(100):
    #     solution = []
    #     totalQuantity = 0
    #     totalDimension = 0
    #     for j in range(len(productsList)):
    #         quantity = random.randint(0, maxProductQuantity)
    #         totalQuantity += quantity
    #         totalDimension += quantity * productsList[j][2]
    #         solution.append(quantity)
    #     if totalQuantity <= maxTotalQuantity and totalDimension <= storeDimension:
    #         population.append(solution)

    #represents code above
    firstpopulation(productsList)

    # Set the number of generations and the mutation rate
    numGenerations = 1000
    mutationRate = 0.01

    # Run the genetic algorithm
    for generation in range(numGenerations):
        # Calculate the fitness of each solution in the population
        populationFitness = []
        for i in range(len(population)):
            totalProfit = 0
            totalQuantity = 0
            totalDimension = 0
            for j in range(len(productsList)):
                totalProfit += population[i][j] * productsList[j][2]
                totalQuantity += population[i][j]
                totalDimension += population[i][j] * productsList[j][3]
            if totalQuantity <= maxTotalQuantity and totalDimension <= storeDimension:
                populationFitness.append(totalProfit)
            else:
                populationFitness.append(0)

        # Select the fittest solutions for breeding
        fittestSolutions = []
        for i in range(int(len(population) / 2)):
            fittestIndex = populationFitness.index(max(populationFitness))
            fittestSolutions.append(population[fittestIndex])
            populationFitness[fittestIndex] = 0
        population = fittestSolutions

        # Breed the solutions to create a new population
        newPopulation = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[random.randint(0, len(population) - 1)]
            child = []
            for j in range(len(productsList)):
                if random.random() < 0.5:
                    child.append(parent1[j])
                else:
                    child.append(parent2[j])

            # Mutate the child
            if random.random() < mutationRate:
                mutationIndex = random.randint(0, len(productsList) - 1)
                child[mutationIndex] = random.randint(0, maxProductQuantity)

            newPopulation.append(child)
        population = newPopulation
        print(generation)
    # Return the fittest solution

    return population[0]

In [223]:
optimize_product_quantities(productsList,maxTotalAmount,maxAmountProduct,storeDimension)

UnboundLocalError: local variable 'population' referenced before assignment

In [185]:


# Define the fitness function
def fitness(x):
    # x is the number of products to buy
    # We can define the fitness as the difference between the desired stock level and the actual stock level
    # The desired stock level is 10
    return abs(10 - x)

# Initialize the population with random solutions
population = [random.randint(0, 20) for _ in range(10)]

# Set the maximum number of generations
max_generations = 100

# Iterate over the generations
for generation in range(max_generations):
    # Evaluate the fitness of each solution
    fitness_values = [fitness(x) for x in population]
    
    # Select the best solutions as parents
    # We'll use tournament selection here, which involves selecting a random subset of the population
    # and choosing the best solution from that subset as the parent
    parent1 = min(random.sample(population, 5), key=fitness)
    parent2 = min(random.sample(population, 5), key=fitness)
    
    # Create offspring by combining the parents using crossover
    # We'll use one-point crossover here, which involves selecting a random point in the parents' genomes
    # and swapping the genes on either side of the point to create the offspring
    crossover_point = random.randint(1, len(parent1))
    offspring = parent1[:crossover_point] + parent2[crossover_point:]
    
    # Add some randomness to the offspring using mutation
    # We'll flip a random bit in the offspring's genome
    mutation_point = random.randint(0, len(offspring) - 1)
    offspring[mutation_point] = 1 - offspring[mutation_point]
    
    # Replace the worst solution in the population with the offspring
    worst_solution = max(zip(population, fitness_values), key=lambda x: x[1])
    population[population.index(worst_solution[0])] = offspring

# The final solution is the best solution in the final generation
best_solution = min(zip(population, fitness_values), key=lambda x: x[1])
print(f"The optimal number of products to buy is: {best_solution[0]}")

TypeError: object of type 'int' has no len()

In [ ]:

  # evaluate the fitness of a given solution
  def evaluate_fitness(products):

#  # check if better (e.g. perform a tournament)
#  if scores[ix] < scores[selection_ix]:
#  selection_ix = ix
#  return pop[selection_ix]
#{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0},  

    bestQuantity = 0
        if products['predictedProfits'] < products['predictedProductQuantities']:
      return 0

    
    total_dimension = sum([q * d for q, d in zip(solution, product_dimensions)])
    if total_quantity < min_quantity or total_quantity > max_quantity or total_dimension > store_dimension:
      return 0
    profit = calculate_profit(solution)
    # calculate the deviation from the target profit
    deviation = abs(profit - target_profit)
    # the fitness is higher for solutions that have a lower deviation from the target profit
    return 1 / (deviation + 1)

  # genetic algorithm function
  def genetic_algorithm(fitness_fn):
    # initialize population
    population = initialize_population(product_quantities)
    for generation in range(max_generations):
      # evaluate fitness
      fitnesses = [fitness_fn(s) for s in population]
      # select parents
      parents = select_parents(population, fitnesses)
      # create offspring
      offspring = create_offspring(parents)
      # mutate offspring
      offspring = mutate(offspring)
      # replace least fit individuals in population with offspring
      population = replace_least_fit(population, offspring)
    # return the fittest solution
    return population[0]

  # use the genetic algorithm to find the optimal solution
  return genetic_algorithm(evaluate_fitness)

In [ ]:
#--------------------------------------------------------------------------
